In [1]:
import numpy as np
import scann

2023-10-18 12:55:24.548693: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-18 12:55:24.584690: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-18 12:55:24.585536: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 12:55:25.237115: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [41]:
x = np.array([
       np.array([
          np.random.normal(10, 1),
          np.random.normal( 0, 1),
          np.random.normal( 0, 1)
     ])
     for _ in range(1000000)
   ]).astype(np.float32)

y = np.array([
       np.array([
          np.random.normal( 0, 1),
          np.random.normal(10, 1),
          np.random.normal( 0, 1)
     ])
     for _ in range(1000000)
   ]).astype(np.float32)

z = np.array([
       np.array([
          np.random.normal( 0, 1),
          np.random.normal( 0, 1),
          np.random.normal(10, 1)
     ])
     for _ in range(1000000)
   ]).astype(np.float32)
    

In [13]:
needle = np.array([np.array([
     np.random.normal( 5, 1),
     np.random.normal( 5, 1),
     np.random.normal( 5, 1)
])])

In [42]:
needles = np.array([
       np.array([
          np.random.normal( 5, 1),
          np.random.normal( 5, 1),
          np.random.normal( 5, 1)
     ])
     for _ in range(1000)
   ]).astype(np.float32)

In [35]:
x

array([[10.042362  , -0.5758852 , -0.37699685],
       [ 9.631416  , -0.5559351 , -0.12972423],
       [10.329786  ,  0.2505935 , -0.8420398 ],
       ...,
       [11.015576  ,  0.32923988, -1.4416692 ],
       [ 9.895424  ,  1.11646   ,  2.2240582 ],
       [ 8.93433   , -1.6914378 ,  1.4024101 ]], dtype=float32)

In [43]:
needles

array([[6.2346635, 3.3157349, 4.383872 ],
       [4.101266 , 5.078736 , 4.376109 ],
       [5.1757627, 3.2648637, 4.5233755],
       ...,
       [5.3403163, 4.5867066, 3.6266282],
       [5.454832 , 4.9187737, 5.3977494],
       [5.526165 , 4.3032765, 5.9684315]], dtype=float32)

In [44]:
data = np.concatenate( (x, y, needles, z) )

In [45]:
searcher = scann.scann_ops_pybind.builder(
    data,                             # numpy array containing the data points to be indexed
    num_neighbors        =  20,              # the number of neighbors to search for
    distance_measure     = "squared_l2"      # the distance metric to use (e.g., 'dot_product' or 'squared_l2')
).tree(                                  # .tree() is a method that can be used after(?) calling .build() to further refine the tree structure of the index. .tree() allows us to fine-tune the tree structure of the index to optimize the search performance for a given dataset and use case. It takes several arguments: 
    num_leaves           =  2000,          # num_leaves: the maximum number of leaves in the tree . This can be used to control the balance between search speed and memory usage. Increasing num_leaves will improve the search accuracy but also increase the memory usage.
    num_leaves_to_search =   100,          #num_leaves_to_search: the number of leaves to search during the query phase
    training_sample_size = 250000           # training_sample_size: the number of vectors to use for training the tree.
).score_ah(                              # score_ah() is a method in ScaNN that sets the parameters for the asymmetric hash function used in the indexing process. It is called after .tree() because the tree structure is required to set these parameters.
    2,                                   # num_neighbors sets the number of neighbors to consider when selecting the threshold for asymmetric hashing   
    anisotropic_quantization_threshold = 0.2 # anisotropic_quantization_threshold controls how many different quantization levels to use for each dimension in the hash function (maller values generally resulting in better accuracy at the cost of higher indexing time and memory usage.)
).reorder(
    5
).build()  # Finally, the ScannBuilder object's build() method is called to create the index.

2023-10-18 13:02:55.507881: I scann/partitioning/partitioner_factory_base.cc:59] Size of sampled dataset for training partition: 249914
2023-10-18 13:02:56.651966: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:84] PartitionerFactory ran in 1.144004182s.


In [46]:
query = np.array([ 5.0, 5.0, 5.0 ]).astype(np.float32)

In [47]:
# neighbours is an array of indexes for nearest neighbors. distances is an array with their corresponding squared_L2 distances
neighbors, distances = searcher.search(query, final_num_neighbors=30)

In [48]:
len(neighbors)

5

In [49]:
for x in zip(neighbors,distances): print(x)

(2000005, 1.470756)
(2000004, 1.5226725)
(2000003, 1.5736524)
(2000002, 3.2687614)
(2000000, 4.740757)
